# 7.2 BICの計算

本ファイルは、7.2節の実装です。

データに対してBICの値を求めます。

## プログラム実行前の設定など

In [0]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)


In [0]:
# 使用するパッケージ（ライブラリと関数）を定義
from numpy.random import *
import pandas as pd


## データの作成

In [3]:
# データ数
num_data = 10

# x1：0か1の値をnum_data個生成、0の確率は0.6、1の確率は0.4
x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])

# x2：0か1の値をnum_data個生成、0の確率は0.4、1の確率は0.6
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

# 2変数で表にする
df = pd.DataFrame({'x1': x1,
                   'x2': x2,
                   })

df.head()  # 先頭を表示


,x1,x2
0,0,0
1,1,1
2,0,1
3,1,1
4,1,0


In [4]:
# 変数x3：0か1の値をnum_data個生成する
# (x1,x2)= (0,0)のとき、0の確率は0.2
# (x1,x2)= (1,0)のとき、0の確率は0.3
# (x1,x2)= (0,1)のとき、0の確率は0.4
# (x1,x2)= (1,1)のとき、0の確率は0.1

x3 = []
for i in range(num_data):
  if x1[i] == 0 and x2[i] == 0:
    x3_value = np.random.choice([0, 1], 1, p=[0.2, 0.8])
    x3.append(x3_value[0])  # x3はリストになっているので、0番目の要素を取り出して追加
  elif x1[i] == 0 and x2[i] == 1:
    x3_value = np.random.choice([0, 1], 1, p=[0.3, 0.7])
    x3.append(x3_value[0])
  elif x1[i] == 1 and x2[i] == 0:
    x3_value = np.random.choice([0, 1], 1, p=[0.4, 0.6])
    x3.append(x3_value[0])
  elif x1[i] == 1 and x2[i] == 1:
    x3_value = np.random.choice([0, 1], 1, p=[0.1, 0.9])
    x3.append(x3_value[0])

df["x3"] = x3

df  # 表示


,x1,x2,x3
0,0,0,1
1,1,1,1
2,0,1,0
3,1,1,1
4,1,0,1
5,0,1,1
6,0,1,1
7,1,0,1
8,1,1,1
9,1,1,1


## pgmpy（Python library for Probabilistic Graphical Models）によるBICの計算


In [5]:
!pip install pgmpy==0.1.9

     |████████████████████████████████| 337kB 2.8MB/s 


In [0]:
# 正解のDAGを与える
from pgmpy.models import BayesianModel
model = BayesianModel([('x1', 'x3'), ('x2', 'x3')])  # x1 -> x3 <- x2


In [7]:
# 各データパターンの個数を表示する
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, df)
print("\n", pe.state_counts('x1'))
print("\n", pe.state_counts('x2'))
print("\n", pe.state_counts('x3'))


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



    x1
0   4
1   6

    x2
0   3
1   7

 x1    0         1     
x2    0    1    0    1
x3                    
0   0.0  1.0  0.0  0.0
1   1.0  2.0  2.0  4.0


In [8]:
# CPT（条件付き確率表）を推定する
from pgmpy.estimators import BayesianEstimator

estimator = BayesianEstimator(model, df)

cpd_x1 = estimator.estimate_cpd(
    'x1', prior_type="dirichlet", pseudo_counts=[[0], [0]])
cpd_x2 = estimator.estimate_cpd(
    'x2', prior_type="dirichlet", pseudo_counts=[[0], [0]])
cpd_x3 = estimator.estimate_cpd('x3', prior_type="dirichlet", pseudo_counts=[
                                [0, 0, 0, 0], [0, 0, 0, 0]])
# 注意：pseudo_countsはハイパーパラメータ1のディクレ分布の設定を与えています。

print(cpd_x1)
print(cpd_x2)
print(cpd_x3)


+-------+-----+
| x1(0) | 0.4 |
+-------+-----+
| x1(1) | 0.6 |
+-------+-----+
+-------+-----+
| x2(0) | 0.3 |
+-------+-----+
| x2(1) | 0.7 |
+-------+-----+
+-------+-------+--------------------+-------+-------+
| x1    | x1(0) | x1(0)              | x1(1) | x1(1) |
+-------+-------+--------------------+-------+-------+
| x2    | x2(0) | x2(1)              | x2(0) | x2(1) |
+-------+-------+--------------------+-------+-------+
| x3(0) | 0.0   | 0.3333333333333333 | 0.0   | 0.0   |
+-------+-------+--------------------+-------+-------+
| x3(1) | 1.0   | 0.6666666666666666 | 1.0   | 1.0   |
+-------+-------+--------------------+-------+-------+


In [9]:
# BICを求める
from pgmpy.estimators import BicScore
bic = BicScore(df)
print(bic.score(model))


-21.65605747450808


## 異なるDAGでのBICの計算

In [10]:
# 正解ではないDAGを与える
from pgmpy.models import BayesianModel
model = BayesianModel([('x2', 'x1'), ('x2', 'x3')])  # x1 <- x2 -> x3
bic = BicScore(df)
print(bic.score(model))


-21.425819218840655


以上